In [ ]:
import sketch
import pandas as pd
import pyarrow as pa
import pickle

In [ ]:
import glob
from sketch.sketches import DS_THETA
import datasketches

rowsketches = []
columns = []
for path in glob.glob("../data_examples/tables/*.parquet"):
    df = pd.read_parquet(path)
    for i in range(len(df)):
        test_df = pickle.loads(df['pa_table_pickle'].iloc[i]).to_pandas()
        for column in test_df:
            rowsketches.append(DS_THETA.from_series(test_df[column]).data)
            columns.append(test_df[column])
    if len(rowsketches) > 100000:
        break

In [ ]:
import glob
from sketch.sketches import DS_THETA
import datasketches

rowsketch_binary = []
for path in glob.glob("../data_examples/tables/*.parquet"):
    df = pd.read_parquet(path)
    for i in range(len(df)):
        test_df = pickle.loads(df['pa_table_pickle'].iloc[i]).to_pandas()
        for column in test_df:
            rowsketch_binary.append(DS_THETA.from_series(test_df[column]).to_dict()['data'])
    if len(rowsketches) > 100000:
        break

In [ ]:
import base64
import numpy as np
binarydata = [np.unpackbits(np.frombuffer(base64.b64decode(x), dtype='uint8'))[:1000] for x in rowsketch_binary]
binarydata = [np.pad(x, (0, 1000-len(x))) for x in binarydata]

In [ ]:
binaryversion = pd.DataFrame(np.array(binarydata))

In [ ]:
import umap

fit = umap.UMAP(
    n_components=2,
    n_neighbors=50,
    min_dist=0.01,
    metric="hamming"
)
# u = fit.fit_transform(binaryversion)
u2 = fit.fit(binaryversion)


In [ ]:
import umap.plot


In [ ]:
# umap.plot.points(u2)
umap.plot.output_notebook()


In [ ]:
u2hover = pd.DataFrame([str(x.unique()) for x in columns], columns=["col"])
u2len = pd.DataFrame([len(x.unique()) for x in columns], columns=["colunique"])

In [ ]:
.ndim

In [ ]:
p = umap.plot.points(u2, width=800, height=800)
# values=u2len['colunique'].values
umap.plot.show(p)

In [ ]:
p = umap.plot.interactive(u2, hover_data=u2hover, point_size=5, width=1800, height=1200)
umap.plot.show(p)

In [ ]:
# Create an embedding of every column. Will start with every column getting a HLL sketch

In [ ]:
metric_func = lambda a, b: 1.0-datasketches.theta_jaccard_similarity.jaccard(a, b)[1]

In [ ]:
# let's just brute force this, grab 200 random, then distance to those.
import random
test = random.sample(rowsketches, k=200)

distances = []
for comp in test:
    column = []
    for rs in rowsketches:
        column.append(metric_func(comp, rs))
    distances.append(column)
# (about 20s for 10k x 100)

In [ ]:
data = pd.DataFrame(distances).T
data.shape

In [ ]:
data.min(axis=1)

In [ ]:
import umap

fit = umap.UMAP(
    n_components=2,
    n_neighbors=20,
    min_dist=0.2,
)
u = fit.fit_transform(data)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16, 10))
ax = fig.add_subplot(111)
ax.scatter(u[:,0], u[:,1])
plt.show()

In [ ]:
bounds = [[-19, -18], [-3, -2]]
import numpy as np
indexer = np.logical_and(
    np.logical_and(
        u[:,0] > bounds[0][0],
        u[:,0] < bounds[0][1]),
    np.logical_and(
        u[:,1] > bounds[1][0],
        u[:,1] < bounds[1][1])
)
u2 = u[indexer]
ilocs = np.arange(0, len(u))[indexer]

In [ ]:
# ilocs

In [ ]:
columns[random.choice(ilocs)]

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16, 10))
ax = fig.add_subplot(111)
ax.scatter(u2[:,0], u2[:,1])
plt.show()

In [ ]:
state_in_colname = []
for i, col in enumerate(columns):
    if 'state' in str(col.name).lower():
        state_in_colname.append(i)

In [ ]:
for i in state_in_colname:
    if 'Alaska' in columns[i].values:
        print(i)

In [ ]:
print(u[2918], u[2921], u[83311], u[97020])

In [ ]:
u[state_in_colname[6]]

In [ ]:
u[state_in_colname[7]]

In [ ]:
u[state_in_colname[8]]

In [ ]:
# u3 = u[state_in_colname]
# import matplotlib.pyplot as plt
# fig = plt.figure(figsize=(16, 10))
# ax = fig.add_subplot(111)
# ax.scatter(u3[:,0], u3[:,1])
# plt.show()

In [ ]:
columns[55].name

In [ ]:
datasketches.theta_jaccard_similarity.jaccard(temp.data, temp.data)

In [ ]:
import pickle
test = pickle.loads(df['pa_table_pickle'].iloc[0])

In [ ]:
test.to_pandas()